# 10: Task a asynchronní programování
**autor: Erik Král ekral@utb.cz**

---


Obsah:
- Task
- Klíčové slova async a await
- Asynchronní metody a výjimky.

V následujících kódech budeme používat především ukázky konzolových aplikací, ale ty z principu nejsou asynchroní a uvádíme je jen pro jednoduchost. Typické použítí je ve webových a UI frameworcích.

## Task

Pomocí třídy `Task` můžeme spouštět typicky asynchronně jednu operaci (metodu) bez návratové hodnoty. 

V následujícím příkladu spustíme provádění metody Metoda pomocí statické metody `Task.Run`, tato metoda nám vrátí proměnnou typu `Task`, krerá popisuje již spuštěnou operaci běžící typicky v jiném vlákně. 

V následujícím příkladu nejprve pomocí příkazu `Task.Run(DlouhoBeziciMetoda);` spustíme provádění metody `DlouhoBeziciMetoda`, která vypisuje text na konzoli v intervalu jedné sekundy. A poté začneme souběžně vypisovat v hlavním(main) vlákně ve stejném intervalu také na konzoli. Zápis na konzoli je možný a bezpečný z obou vláken, protože konzole zápis a čtení synchronizuje [(Console I/O Streams. Microsoft Docs. 2022)](https://docs.microsoft.com/en-us/dotnet/api/system.console?view=net-6.0#console-io-streams). 

Příkaz `task.Wait()` na konci čeká na dokončení tasku tak, aby se nám program předčasně nepřerušil před jeho dokončením.

In [7]:
void DlouhoBeziciMetoda()
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);
    }
}

Task task = Task.Run(DlouhoBeziciMetoda);

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

task.Wait();

Main: 0
Metoda: 0
Metoda: 1
Main: 1
Main: 2
Metoda: 2
Main: 3
Metoda: 3
Metoda: 4
Main: 4
Main: 5
Main: 6
Main: 7


Zkuste si v předcházejícím příkladu zvolit kratší interval pro výpis v hlavním vlákně a případně odstranit poslední příkaz task.Wait();

Pokud bychom použili příkaz `task.Wait()` hned po spuštění tasku, tak se běh hlavního vlákna pozastaví dokud nebude task dokončen a výsledkem by bylo, že výpisy neproběhnou souběžně.

In [8]:
void DlouhoBeziciMetoda()
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);
    }
}

Task task = Task.Run(DlouhoBeziciMetoda);

task.Wait();

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}


Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Main: 5
Main: 6
Main: 7


## `Task<T>` s návratovou hodnotou

Pokud metoda vrací hodnotu, tak můžeme využít generický typ `Task<T>`. 

V následujícím příkladu vrací `TDlouhoBeziciMetoda` náhodné číslo po dokončení simulovaného dlouhého výpočtu. Výsledek získáme pomocí příkazu `task.Result`, který opět pozastaví hlavní vlákno, dokud se task nedokončí a nevrátí se výsledek.

In [9]:
int DlouhoBeziciMetoda()
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);
    }

    return Random.Shared.Next(0, 100);
}

Task<int> task = Task.Run(DlouhoBeziciMetoda);

int vysledek = task.Result;

Console.WriteLine($"vysledek: {vysledek}");

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
vysledek: 80
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Main: 5
Main: 6
Main: 7


## Klíčové slova async a await

Pomocí klíčových slov `async` a `await` můžeme jednoduše synchronizovat asynchronní operace a přítom kód zůstal přehledný a mohli jsme používat vyjímky stejným způsobem jako v běžném kódu. Například v programu často chceme stáhnout v seznam produktů v jiném vlákně, tak aby se nám nezablokovalo hlavní vlákno klientské aplikaci a po dokončení stažení produků, chceme tyto produkty zobrazit.

Proto, abychom mohli demonstrovat použití klíčového slova `await` v konzolové aplikaci, tak si vytvoříme složitější příklad. Spuštění tasku je přesunuté přímo do metody `DlouhoBeziciMetodaAsync`, která vrací přímo typ `Task<int>`. Podle konvence má mít název metody, která vrací Task příponu *"Async"*. Potom máme druhou metodu, která čeká na dokončení metody `DlouhoBeziciMetodaAsync` a pak zobrazí výsledek výpočtu.

Nejprve si ale ukážeme příklad bez použití `await` a místo toho použijeme příkaz `task.Result`, což znamená že se se hlavní vlákno pozastaví a kód se neprovádí souběžně. 

In [11]:
Task<int> DlouhoBeziciMetodaAsync()
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);
        }

        return Random.Shared.Next(0, 100); 
    });
}

void MetodaCekajiciNaVysledek()
{
    Task<int> task = DlouhoBeziciMetodaAsync();
    
    int vysledek = task.Result;

    Console.WriteLine($"vysledek: {vysledek}");
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
vysledek: 73
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Main: 5
Main: 6
Main: 7
Konec programu


Nyní použijeme klíčové slovo `await`, díky čemuž se `DlouhoBeziciMetodaAsync` provádí souběžně s kódem v hlavním vlákně a po jejím dokončení se hlavní vlákno přeruší, provede se kód za příkazem `await` a pak zase pokračuje hlavní vlákno. Klíčové slovo `await` můžeme použít jen v metodě označené klíčovým slovem `async`. Místo návratového typu `void` u asynchronní metody používáme `Task`. 

In [12]:
Task<int> DlouhoBeziciMetodaAsync()
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);
        }

        return Random.Shared.Next(0, 100); 
    });
}

async Task MetodaCekajiciNaVysledekAsync()
{
    Task<int> task = DlouhoBeziciMetodaAsync();
    
    int vysledek = await task;

    Console.WriteLine($"vysledek: {vysledek}");
}

Task task = MetodaCekajiciNaVysledekAsync();

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Metoda: 0
Main: 0
Main: 1
Metoda: 1
Metoda: 2
Main: 2
Main: 3
Metoda: 3
Main: 4
Metoda: 4
Main: 5
vysledek: 32
Main: 6
Main: 7
Konec programu


## Návratový typ funkcí označených async

### Asynchronní metoda s návratovým typem void

V přechozím příkladu jsme u `async` metody použili návratový typ `Task`. V následujících kódu si na zjednodušených příkladech ukážeme různé varianty návratových typů.

Bylo by sice možné použít `void`:

In [ ]:
Task<int> MetodaAsync()
{
    return Task.FromResult<int>(1);
}

async void DruhaMetoda()
{
    int vysledek = await MetodaAsync();
}

Ale poté bychom nemohli použít `await` a také bychom nemohli v takové metodě ošetřovat jednoduše výjimky. Možnost mít návratový typ void byla přidaná kvůli UI Frameworkům Winform a WPF pro handlery eventů a neměla by se běžně používat. [How Async/Await Really Works in C#](https://devblogs.microsoft.com/dotnet/how-async-await-really-works/).

Následující kód by měl vyvolat výjimku, ale v konzolové aplikaci ji nevyvolá. Ve frameworcích Winform, WPF nebo AvaloniaUI by to mohlo být jinak.

In [13]:
Task<int> MetodaAsync()
{
    throw new Exception();
}

async void DruhaMetoda()
{
    int vysledek = await MetodaAsync();
}

DruhaMetoda();

Pokud použijeme návratový typ `Task<int>`, tak můžeme zachytit výjimku.

In [1]:
Task<int> MetodaAsync()
{
    throw new Exception();
}

async Task DruhaMetodaAsync()
{
    int vysledek = await MetodaAsync();
}

await DruhaMetodaAsync();

Error: System.Exception: Exception of type 'System.Exception' was thrown.
   at Submission#2.MetodaAsync()
   at Submission#2.<DruhaMetodaAsync>d__2.MoveNext()
--- End of stack trace from previous location ---
   at Submission#2.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Přechozí kód vyvolal výjimku, což je správné chování.

Pokud metoda vrací hodnotu, tak opět použijeme generický Task<T>.

In [ ]:
Task<int> MetodaAsync()
{
    return Task.FromResult<int>(1);
}

async Task<int> DruhaMetodaAsync()
{
    int vysledek = await MetodaAsync();

    return vysledek;
}

int vysledek = await DruhaMetodaAsync();

## Příklad webová služba

In [2]:
using System.Net.Http;

using (HttpClient client = new HttpClient())
{
    try
    {
        string  vtip = await client.GetStringAsync("https://geek-jokes.sameerkumar.website/api?format=text");
        
        Console.WriteLine(System.Web.HttpUtility.HtmlDecode(vtip));
    }
    catch (HttpRequestException ex)
    {
        System.Diagnostics.Debug.WriteLine(ex.Message);
        throw;
    }
}

"Chuck Norris once shot down a German fighter plane with his finger. By yelling 'Bang!'"



## Úkol desktopová aplikace

Do následující desktopové aplikace přidejte předchozí načítání vtipu. Aplikace by měla jít spustit z tohoto notebooky na Windows. 

Pokud máte MacOs nebo Unix, tak projekt najdete zde zde:

[Notebooks/OPN_10_AvaloniaProject](https://github.com/ekral/FAI/tree/master/OP/Notebooks/OPN_10_AvaloniaProject)

Nebo si přímo můžete stáhnout zip s projektem:

[Download-directory.github.io](https://download-directory.github.io/?url=https%3A%2F%2Fgithub.com%2Fekral%2FFAI%2Ftree%2Fmaster%2FOP%2FNotebooks%2FOPN_10_AvaloniaProject)



Nejprve si načteme nuget balíček pro UI framework Avalonia.

In [3]:
#r "nuget: Avalonia.Desktop"

using Avalonia;
using Avalonia.Controls;
using Avalonia.Controls.ApplicationLifetimes;
using Avalonia.Layout;
using Avalonia.Threading;
using System.Threading;

Installed Packages Avalonia.Desktop, 0.10.19

Loading extensions from `C:\Users\ekral\.nuget\packages\skiasharp\2.88.1-preview.108\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

Následující kód inicializuje UI framework, tento kód spusťe **jen jednou**. Pokud byste spustili vícekrát, tak proveďte Restart kernelu a poté kód spuťte znovu.

In [4]:
var tcs = new TaskCompletionSource<SynchronizationContext>();

AppBuilder.Configure<Application>()
    .UsePlatformDetect()
    .AfterSetup(builder => tcs.SetResult(SynchronizationContext.Current))
    .SetupWithLifetime(new ClassicDesktopStyleApplicationLifetime());

Application.Current.Styles.Add(new Avalonia.Themes.Fluent.FluentTheme(new Uri("avares://MyAssembly")) 
{ 
    Mode = Avalonia.Themes.Fluent.FluentThemeMode.Dark
});

Zde je vlastní definice okna aplikace, kterou změňte dle zadání:

In [13]:
class MyWindow : Window
{
    Task<int> DlouhoBeziciMetodaAsync()
    {
        return Task.Run(() =>
        {
            System.Threading.Thread.Sleep(4000);
            return Random.Shared.Next(0, 100);
        });
    }

    public MyWindow()
    {
        Title = "AvaloniaUI";

        Button button = new Button() 
        { 
            HorizontalAlignment = HorizontalAlignment.Center,
            Content = "Click" 
        };

        TextBlock textBlock = new TextBlock() 
        { 
            HorizontalAlignment = HorizontalAlignment.Center,
            Text = "0" 
        };

        button.Click += async (sender, args) =>
        {            
            int vysledek = await DlouhoBeziciMetodaAsync();
            textBlock.Text = vysledek.ToString();
        };

        Content = new StackPanel() 
        {
            HorizontalAlignment = HorizontalAlignment.Center,
            VerticalAlignment = VerticalAlignment.Center,
            Children = 
            {
                textBlock,
                button
            }
        };
    }   
}

A aplikaci spuťte následujícím příkazem:

In [14]:
SynchronizationContext.SetSynchronizationContext(tcs.Task.Result);

Application.Current.Run(new MyWindow());

---
Více se o typu Task a klíčových slovech můžete dozvědět například zde:

[Task-based asynchronous pattern. Microsoft Docs. 2022](https://docs.microsoft.com/en-us/dotnet/standard/parallel-programming/task-based-asynchronous-programming)

A zde najdete přehled best practice asynchronního programování v jazyce C#:

[Async/Await - Best Practices in Asynchronous Programming. MSDN Magazine Issues. 2013](https://docs.microsoft.com/en-us/archive/msdn-magazine/2013/march/async-await-best-practices-in-asynchronous-programming)

Pro pokročilé:

[How Async/Await Really Works in C#](https://devblogs.microsoft.com/dotnet/how-async-await-really-works/)
